<a href="https://colab.research.google.com/github/biruk50/Medium_articles/blob/main/negative_cycle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import heapq
from collections import deque, defaultdict
import time


def generate_complete_graph(n, weight_range=(-2, 45)):
    graph = defaultdict(list)
    min_edge_weight = [float('inf')] * n
    out_in = [0] * n

    for u in range(n):
        for v in range(n):
            if u != v:
                weight = random.randint(*weight_range)
                graph[u].append((v, weight))
                out_in[u] += 1
                out_in[v] -= 1

                if weight < min_edge_weight[u]:
                    min_edge_weight[u] = weight

    return graph, min_edge_weight, out_in

# SPFA algorithm
def spfa(graph, source, n):
    dist = [float('inf')] * n
    in_queue = [False] * n
    dist[source] = 0
    q = deque([source])
    in_queue[source] = True

    count = [0] * n
    enqueue_count = 0

    while q:
        u = q.popleft()
        in_queue[u] = False
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                if not in_queue[v]:
                    q.append(v)
                    in_queue[v] = True
                count[v] += 1
                enqueue_count += 1
                if count[v] > n:
                    print("SPFA: Negative cycle detected.")
                    return None, enqueue_count
    return dist, enqueue_count

# Custom SPFA-like algorithm with heap based on min edge and imbalance

def custom_spfa_heap(graph, source, n, min_edge_weight, out_in):
    dist = [float('inf')] * n
    in_heap = [False] * n
    dist[source] = 0
    heap = [(min_edge_weight[source], -out_in[source], source)]
    in_heap[source] = True

    count = [0] * n
    enqueue_count = 0

    while heap:
        _, _, u = heapq.heappop(heap)
        in_heap[u] = False

        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                if not in_heap[v]:
                    heapq.heappush(heap, (min_edge_weight[v], -out_in[v], v))
                    in_heap[v] = True
                count[v] += 1
                enqueue_count += 1
                if count[v] > n:
                    print("Custom SPFA-Heap: Negative cycle detected.")
                    return None, enqueue_count
    return dist, enqueue_count



n = 800  # Number of nodes in complete graph
source = 0
graph, min_weights, imbalance = generate_complete_graph(n)

start = time.time()
spfa_result = spfa(graph, source, n)
spfa_time = time.time() - start

start = time.time()
custom_result = custom_spfa_heap(graph, source, n, min_weights, imbalance)
custom_time = time.time() - start

print("SPFA Result:", spfa_result)
print("Custom SPFA-Heap Result:", custom_result)
print(f"SPFA Time: {spfa_time:.6f} seconds")
print(f"Custom SPFA-Heap Time: {custom_time:.6f} seconds")
